In [ ]:
import pymc as pm # switch to pymc_env for training
from kneed import KneeLocator
import json

from train.utils_perx import *
from models.types import *
from models.import_plotting import *


METRIC = "m" # {"m", "p75", "p95"}
applications = APPLICATIONS_E_GPU

DATA_BASE_FOLDER = "server1_cpu_base"
DATA_FOLDER = f"{HOME_PATH}/server1_cpu_pair"
OUT_FOLDER_PERX = f"{HOME_PATH}/server_1/results/models_cpu/models_{METRIC}/perx"
OUT_FOLDER_SCALORAN = f"{HOME_PATH}/server_2/results/models_{METRIC}/scaloran"

if not os.path.exists(OUT_FOLDER_PERX):
    os.makedirs(OUT_FOLDER_PERX)

if not os.path.exists(OUT_FOLDER_SCALORAN):
    os.makedirs(OUT_FOLDER_SCALORAN)

In [ ]:
baselines = {}
for app in applications:
    df = pd.read_csv(os.path.join(DATA_BASE_FOLDER, app, "tenant-1.csv"))
    df.dropna(inplace=True)
    match METRIC:
        case "m":
            sts = np.mean(df["service_times"])
        case "p75":
            sts = np.percentile(df["service_times"], 75)
        case "p95":
            sts = np.percentile(df["service_times"], 95)
    
    baselines[app] = sts

json.dump(baselines, open(os.path.join(OUT_FOLDER_PERX, "baselines.json"), "w"), indent=4)

In [ ]:
class BayesianEstParam:
    def __init__(self):
        pass

    def fit_on_data(self, coords, n_idx, x, y):
        self.model = pm.Model(coords=coords)
        with self.model:
            group_idx = pm.MutableData("group_idx", n_idx, dims="obs_id")

            alpha1 = pm.Normal("alpha1", mu=0, sigma=2.5)
            beta1 = pm.Normal("beta1", mu=0, sigma=2.5)
            # theta1 = alpha1 + beta1 * n_idx
            theta1 = pm.Deterministic("theta1", alpha1 + beta1 * n_idx)

            alpha2 = pm.Normal("alpha2", mu=0, sigma=2.5)
            beta2 = pm.Normal("beta2", mu=0, sigma=2.5)
            # theta2 = alpha2 + beta2 * n_idx
            theta2 = pm.Deterministic("theta2", alpha2 + beta2 * n_idx)

            alpha3 = pm.Normal("alpha3", mu=0, sigma=2.5)
            beta3 = pm.Normal("beta3", mu=0, sigma=2.5)
            # theta3 = alpha3 + beta3 * n_idx
            theta3 = pm.Deterministic("theta3", alpha3 + beta3 * n_idx)

            mu = theta1 / (1 + pm.math.exp(-theta2 * (x - theta3)))

            sigma = pm.HalfNormal("sigma", sigma=1.0)
            y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)

            # Find the MAP estimate
            map_ests = pm.find_MAP()

            self.map_ests = map_ests

            return map_ests

In [ ]:
sig_params: Dict[str, Dict] = defaultdict(lambda: {})


for app1 in applications:
    for app2 in applications:

        group_data_est = {
            "n_values": [],
            "x_data": [],
            "y_data": [],
            "n_idx": [],
        }

        sts_df = get_p95_app1_app2_all_ten0(app1, app2, DATA_FOLDER)

        sts_df_group_ni = sts_df.groupby(["ni"])

        for ix, (ni, group_ni) in enumerate(sts_df_group_ni):
            x = group_ni["lam"]
            y = group_ni["sts"]

            group_data_est["x_data"].append(np.array(x))
            group_data_est["y_data"].append(np.array(y))
            group_data_est["n_idx"].append([[ni] * len(x)])
            if ni not in group_data_est["n_values"]:
                group_data_est["n_values"].append(ni)

        group_data_est["x_data"] = np.concatenate(group_data_est["x_data"])
        group_data_est["y_data"] = np.concatenate(group_data_est["y_data"])
        group_data_est["n_idx"] = np.hstack(group_data_est["n_idx"]).reshape((-1,))

        estimator = BayesianEstParam()
        coords = {
            "group": group_data_est["n_values"],
            "obs_id": np.arange(len(group_data_est["x_data"])),
        }

        map_ests = estimator.fit_on_data(
            coords,
            n_idx=group_data_est["n_idx"],
            x=group_data_est["x_data"],
            y=group_data_est["y_data"],
        )

        params = {
            "alpha1": float(estimator.map_ests["alpha1"]),
            "beta1": float(estimator.map_ests["beta1"]),
            "alpha2": float(estimator.map_ests["alpha2"]),
            "beta2": float(estimator.map_ests["beta2"]),
            "alpha3": float(estimator.map_ests["alpha3"]),
            "beta3": float(estimator.map_ests["beta3"]),
        }

        sig_params[app1][app2] = params

In [ ]:
if not os.path.exists(OUT_FOLDER_PERX):
    os.mkdir(OUT_FOLDER_PERX)

json.dump(sig_params, fp=open(os.path.join(OUT_FOLDER_PERX, "sig_params.json"), "w"), indent=4)

## Test Plot


In [ ]:
import json

sig_params = json.load(
    fp=open(os.path.join(OUT_FOLDER_PERX, "sig_params.json"), "r")
)

In [ ]:
plt.figure()

app1 = "iclf-mnet"
app2 = "iclf-mnet"

lam =50.0

sts_df = get_p95_app1_app2_all_ten0(app1, app2, DATA_FOLDER, metric=METRIC)

nis, y = get_ni_sts_curve_from_sig_params(sig_params[app1][app2], lam=lam, ni_start=2, ni_end=32)

plt.plot(nis, y, color="red", label="Est.")

sts_df_group_lam = sts_df.groupby(by="lam")
sts_df_lam = sts_df_group_lam.get_group(lam)

sns.lineplot(sts_df_lam, x="ni", y="sts", color="blue", label="True")
plt.xlabel(get_label("num_instances"))
plt.ylabel(get_label("inf_time"))
plt.grid()
plt.legend()

# save_fig(name="sig_param_mvit_bert")

plt.show()

### Test Elbow Param


In [ ]:
from opt.opt_perx import ElbowPredictor

app = "iclf-mnet"

app_rate = 10.0
a1 = sig_params[app][app]["alpha1"]
b1 = sig_params[app][app]["beta1"]
a2 = sig_params[app][app]["alpha2"]
b2 = sig_params[app][app]["beta2"]
a3 = sig_params[app][app]["alpha3"]
b3 = sig_params[app][app]["beta3"]

nis = list(range(4, 16, 2))
y = []
for ni in nis:
    t1 = a1 + b1 * ni
    t2 = a2 + b2 * ni
    t3 = a3 + b3 * ni

    sig = t1 / (1 + np.exp(-t2 * (app_rate - t3)))
    y.append(sig)

elbow_predictor = ElbowPredictor(nis, y)
yp = []

for ni in nis:
    yp.append(elbow_predictor.predict(ni))

elbow_x = float(elbow_predictor.elbow_point)
elbow_y = float(elbow_predictor.elbow_point_y)

c1 = float(elbow_predictor.lr1.intercept_[0])
m1 = float(elbow_predictor.lr1.coef_[0][0])

x_vals = [4.0]
y_vals = [m1 * 4 + c1]

if elbow_x > 4.0:
    x_vals.append(elbow_x)
    y_vals.append(elbow_y)

if elbow_x != 16 and elbow_predictor.lr2 is not None:
    c2 = float(elbow_predictor.lr2.intercept_[0])
    m2 = float(elbow_predictor.lr2.coef_[0][0])
    x_vals.append(16.0)
    y_vals.append(m2 * 16 + c2)
else:
    x_vals.append(16.0)
    y_vals.append(m1 * 16 + c1)

x_vals, y_vals = [4.0, 15.0, 16.0], [0.35335941266592297, 0.6904434462136148, 0.6904434462136148]

x0, x1 = x_vals[0], x_vals[1]
y0, y1 = y_vals[0], y_vals[1]

# Compute slope (m) and intercept (b) of the first segment
m = (y1 - y0) / (x1 - x0)
b = y0 - m * x0

# Compute extrapolated y at x = 0
y_extrapolated = m * 0 + b

# Prepend extrapolated point
x_vals_extended = [0] + x_vals
y_vals_extended = [y_extrapolated] + y_vals

# Plot
plt.plot(
    x_vals_extended,
    y_vals_extended,
    # marker="o",
    linestyle="-",
    color="b",
    label="PWL with Extrapolation",
)
plt.scatter([0], [y_extrapolated], color="r", label="Extrapolated Point (x=0)")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Piecewise Linear Function with Extrapolation to x = 0")
plt.grid(True)
plt.legend()
# plt.show()